In [22]:
import math
import shutil
from pathlib import Path
from uuid import uuid4

import pandas as pd
from slugify import slugify
# go to https://ford-parts.com.ru/bitrix/admin/iblock_data_export.php?lang=ru
# Каталог -> Каталог товаров и Торговые предложения
# Вытащить категории, сабкатегории и скрестить с новой версией в Postgres
#

In [23]:
catalogue = pd.read_csv("data/category.csv")

catalogue = catalogue[
    (catalogue["IE_NAME"] != "1")
    & (catalogue["IE_NAME"] != "0")
    & (catalogue["IE_NAME"] != "01")
    & (catalogue["IC_GROUP1"].notna())
    & (catalogue["IC_GROUP2"].notna())
]

In [24]:
catalogue = catalogue[
    [
        "IE_XML_ID",
        "IE_NAME",
        "IE_PREVIEW_PICTURE",
        "IE_PREVIEW_TEXT",
        "IC_GROUP1",
        "IC_GROUP2",
    ]
].rename(
    columns={
        "IE_XML_ID": "bitrix_id",
        "IE_NAME": "name",
        "IE_PREVIEW_PICTURE": "old_image",
        "IE_PREVIEW_TEXT": "cross_number",
        "IC_GROUP1": "category",
        "IC_GROUP2": "sub_category",
    }
)
ids = [str(uuid4()) for i in range(len(catalogue))]
catalogue["id"] = ids
catalogue["slug"] = catalogue["name"]
catalogue["slug"] = [slugify(n, lowercase=True) for n in catalogue["slug"]]

In [25]:
# catalogue["url"] = (
#     "https://ford-parts.com.ru/catalog/" + catalogue["bitrix_id"].astype(str) + "/"
# )
# catalogue[
#     catalogue["slug"].isin(catalogue["slug"].value_counts()[:25].index)
# ].sort_values("name").to_csv("seo_duplicates.csv", index=False)

In [26]:
catalogue

,bitrix_id,name,old_image,cross_number,category,sub_category,id,slug
1013,F8-2,Focus II-III / C-Max / Kuga / Mondeo / S-Max 0...,/upload/iblock/965/h6v1v60gi1ryckdh0tc7ay6ltnz...,"1930055, 1743698, 8M51R442A66EA",Кузов,Механизмы замков,ab4409a7-01c9-4de7-96af-b4a599eb27e4,focus-ii-iii-c-max-kuga-mondeo-s-max-07-zamok-...
1014,F8-3,Focus II-III / C-Max / Kuga / Mondeo / S-Max 0...,/upload/iblock/da3/klvvib5jemjl1rp5rfraxiri73a...,"1859161, 8M51R442A66AC, 8M51 R442A66 DB, 15704...",Кузов,Механизмы замков,991b3fe0-81c9-4f8f-947e-09b5809ab3e9,focus-ii-iii-c-max-kuga-mondeo-s-max-07-zamok-...
1015,F8-6,Механизм замка Connect задней (высокой) двери ...,NaN,"1419513, 2T14 V43287 BP",Кузов,Механизмы замков,e62b1641-4bb8-4bd4-bd98-5a443a22d6fc,mekhanizm-zamka-connect-zadnei-vysokoi-dveri-v...
1016,F8-7,Механизм замка L ЛЕВОЙ выдвижной длинной двер...,/upload/iblock/3bc/4b6qcru899yl23shyk2ndyfvic7...,"1766561, YC15-V264A33-BK",Кузов,Механизмы замков,597c156f-d5f8-40f4-87b8-190bee5e3845,mekhanizm-zamka-l-levoi-vydvizhnoi-dlinnoi-dve...
1017,F8-8,Механизм замка R правой выдвижной длинной двер...,/upload/iblock/f18/n4qjnqyb01i5awupzy87xbd8w15...,YC15 B264A00 BL N - FSE 11-321-050 - 1C15 B264...,Кузов,Механизмы замков,e8110a1c-3a13-4adc-ab2f-a41cb962c3db,mekhanizm-zamka-r-pravoi-vydvizhnoi-dlinnoi-dv...
...,...,...,...,...,...,...,...,...
9655,F22-467,Датчик давления выхлопных газов Ford Transit 2...,NaN,"1600077, 6G9N-5L200-AB",Система топливная,Датчик давления воздуха (MAP / наддув / выхлоп),af78417e-10fa-414d-83df-a46f3400d246,datchik-davleniia-vykhlopnykh-gazov-ford-trans...
9656,F22-474,Датчик давления наддува FORD 1.6 TDCI,NaN,"1256481, 1468879, 1231463, 1256481, MAZDA Y601...",Система топливная,Датчик давления воздуха (MAP / наддув / выхлоп),6d630c3e-9284-4da0-8919-e0948a8b3140,datchik-davleniia-nadduva-ford-1-6-tdci
9657,F22-300,Датчик давления фреона Mondeo 1993-2000 / Tran...,/upload/iblock/dd3/dlji06uet8z2kkjp92a1r03qjhy...,"95BW19E561AA, 3317869, 2S6H19E561AA, 1217951, ...",Электрооборудование,Датчики кондиционера,78ec96c5-37c5-4597-b389-fbcfac631ec5,datchik-davleniia-freona-mondeo-1993-2000-tran...
9658,F22-473,Датчик давления фреона,NaN,"95BW 19E561 AA , 1016565",Электрооборудование,Датчики кондиционера,4b1a2c5d-b4a1-41e5-96c3-13df36ea8e70,datchik-davleniia-freona


## Создать категории и сабкатегории

In [27]:
category_images = pd.read_csv("data/images_categories_tcf.csv")

In [28]:
category = pd.DataFrame(catalogue["category"].drop_duplicates())
category = category.rename(columns={"category": "name"})
ids = [str(uuid4()) for i in range(len(category))]
category["category_id"] = ids
category = category.merge(category_images, on="name", how="left")
category

,name,category_id,slug,old_image
0,Кузов,117ab60e-4800-478e-ba04-d8dc8b227d46,kuzov-ford,/upload/iblock/eb0/a7whmp91qy522054431cyx4dcb1...
1,Двигатель,c771d427-6f41-4ee8-a484-e7c7f5e10a24,dvigatel-ford,/upload/iblock/62b/iuf1syct13rhi50he2p4x3ns068...
2,Аксессуары,29590fd1-80a8-44de-b159-6f0dca380c74,aksessuary-ford,/upload/iblock/4ac/eo5pnechgqjs39tiqk71bruzl5z...
3,Система выхлопная и Турбины,590b9640-e38b-40f7-8f54-602108d91905,sistema-vyhlopnaya-i-turbiny-ford,/upload/iblock/65b/xkh1awpw1dlkna93chdivy7pe05...
4,Подвеска,6b21d3f1-f7b2-40e5-b12f-906dd4f54ce3,podveska-ford,/upload/iblock/ef3/lke05y6nond3gcnsle9xd2q4swb...
5,Система охлаждения,14b97f4e-af40-4e54-a623-3dd140ccd0c4,sistema-ohlazhdeniya-ford,/upload/iblock/b36/3x283iz4lbl98hfjum2sr1fup97...
6,Рулевое управление,1dcdfcf8-2340-4676-8f86-3e7bddf50fb6,rulevoe-upravlenie-ford,/upload/iblock/af1/mlo1135rjsuqv24ijzv18801sns...
7,Система зажигания и управ-я,2817180f-126a-4768-b424-f8e412489d2a,sistema-zazhiganiya-i-uprav-ya-ford,/upload/iblock/da3/tkw3vi1jlgxdxrwb0p35l1ipbdl...
8,Стартеры и их компоненты,03a88661-1d50-44cb-abd2-d60efe56f712,startery-i-ih-komponenty-ford,/upload/iblock/deb/2jk27o1ymwwo1fc0tzllhi184ii...
9,Колеса,fa9d529e-ac63-48b2-9688-52f5db02fd0e,kolesa-ford,/upload/iblock/619/7s44rkdn1v0h0njquwe90pjyma8...


In [29]:
sub_category = pd.DataFrame(catalogue["sub_category"].drop_duplicates())
sub_category = sub_category.rename(columns={"sub_category": "name"})
ids = [str(uuid4()) for i in range(len(sub_category))]
sub_category["sub_category_id"] = ids
sub_category["slug"] = [slugify(n) for n in sub_category["name"].str.lower()]
sub_category = sub_category.merge(
    category_images[["name", "old_image"]], on="name", how="left"
)

In [30]:
one_big_table = catalogue.merge(
    category, left_on="category", right_on="name", how="left"
).merge(sub_category, left_on="sub_category", right_on="name", how="left")

In [31]:
categories = one_big_table[["category_id", "name_y", "slug_y", "old_image_y"]].copy()
categories = categories.rename(
    columns={
        "category_id": "id",
        "name_y": "name",
        "slug_y": "slug",
        "old_image_y": "old_image",
    }
)
categories = categories.drop_duplicates("name")

In [32]:
old_dir = Path("./upload")
new_dir = Path("./categories")
new_dir.mkdir(parents=True, exist_ok=True)


def copy_and_rename_image(row) -> str:
    old_value = row["old_image"]

    # пропуск пустых или невалидных значений
    if (
        not isinstance(old_value, str)
        or not old_value.strip()
        or (isinstance(old_value, float) and math.isnan(old_value))
    ):
        return None

    old_value = "." + old_value

    old_path = Path(old_value)
    print(old_path)
    if not old_path.exists():
        print(f"⚠️ Файл не найден: {old_path}")
        return None

    ext = old_path.suffix.lower()
    new_filename = f"{row['id']}{ext}"
    new_path = new_dir / new_filename

    shutil.copy2(old_path, new_path)
    return new_filename


# применяем к датафрейму
categories["image_url"] = categories.apply(copy_and_rename_image, axis=1)

upload\iblock\eb0\a7whmp91qy522054431cyx4dcb1ffcus.PNG
upload\iblock\62b\iuf1syct13rhi50he2p4x3ns0688u5rb.png
upload\iblock\4ac\eo5pnechgqjs39tiqk71bruzl5zyqifl.PNG
upload\iblock\65b\xkh1awpw1dlkna93chdivy7pe052l7y6.PNG
upload\iblock\ef3\lke05y6nond3gcnsle9xd2q4swb2xi3p.PNG
upload\iblock\b36\3x283iz4lbl98hfjum2sr1fup979ghnr.PNG
upload\iblock\af1\mlo1135rjsuqv24ijzv18801snshs6ca.PNG
upload\iblock\da3\tkw3vi1jlgxdxrwb0p35l1ipbdlfopm9.PNG
upload\iblock\deb\2jk27o1ymwwo1fc0tzllhi184iix2o0v.PNG
upload\iblock\619\7s44rkdn1v0h0njquwe90pjyma8pbd46.PNG
upload\iblock\68e\ug1toh7bqcxmvezcg82ozdy2s3zte2p8.PNG
upload\iblock\f79\vkmipce0ynbmq18cjmfmook8w8owz898.PNG
upload\iblock\e67\ib6btlm17dy7fesheum3lmnrsnj4z3q7.png
upload\iblock\72f\ihzz2seihlpibgr6dley1eq8tsu1c1qt.PNG
upload\iblock\f42\rl8be0o3mmipdcvn2owatz1l47opw2ec.PNG
upload\iblock\b9c\ziq173396tc23hvbzf020ax4fq6hu22w.PNG
upload\iblock\b6a\l5v2h0rsfnw11asi9jwmdn4ytdgykn3p\Топливная_система.jpg
upload\iblock\3bb\wi5n804o3qdhwgfp09ll3eeh75ldl

In [33]:
categories["image_url"] = (
    "https://storage.yandexcloud.net/tcf-images/categories/" + categories["image_url"]
)
categories["image_url"] = categories["image_url"].fillna(
    "https://storage.yandexcloud.net/tcf-images/default.svg"
)
categories.drop(columns=["old_image"]).to_csv("out/categories.csv", index=False)

In [34]:
sub_categories = one_big_table[
    ["sub_category_id", "category_id", "name", "slug", "old_image"]
].copy()
sub_categories = sub_categories.rename(
    columns={
        "sub_category_id": "id",
        "name": "name",
        "slug": "slug",
        "old_image": "old_image",
    }
)

In [35]:
old_dir = Path("./upload")
new_dir = Path("./sub_categories")
new_dir.mkdir(parents=True, exist_ok=True)


def copy_and_rename_image(row) -> str:
    old_value = row["old_image"]

    # пропуск пустых или невалидных значений
    if (
        not isinstance(old_value, str)
        or not old_value.strip()
        or (isinstance(old_value, float) and math.isnan(old_value))
    ):
        return None

    old_value = "." + old_value

    old_path = Path(old_value)
    print(old_path)
    if not old_path.exists():
        print(f"⚠️ Файл не найден: {old_path}")
        return None

    ext = old_path.suffix.lower()
    new_filename = f"{row['id']}{ext}"
    new_path = new_dir / new_filename

    shutil.copy2(old_path, new_path)
    return new_filename


# применяем к датафрейму
sub_categories["image_url"] = sub_categories.apply(copy_and_rename_image, axis=1)

upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\iblock\1bd\17v2do3px7367ok0ddt1qqlthzxublyb.png
upload\ibl

In [36]:
sub_categories["image_url"] = (
    "https://storage.yandexcloud.net/tcf-images/sub_categories/"
    + sub_categories["image_url"]
)
sub_categories["image_url"] = sub_categories["image_url"].fillna(
    "https://storage.yandexcloud.net/tcf-images/default.svg"
)
sub_categories.drop(columns=["old_image"]).drop_duplicates("name").to_csv(
    "out/sub_categories.csv", index=False
)

In [37]:
## PRODUCTS

In [38]:
products = one_big_table[
    [
        "bitrix_id",
        "id",
        "sub_category_id",
        "name_x",
        "slug_x",
        "cross_number",
        "old_image_x",
    ]
].copy()
products = products.rename(
    columns={"name_x": "name", "slug_x": "slug", "old_image_x": "old_image"}
)

In [39]:
old_dir = Path("./upload")
new_dir = Path("./products")
new_dir.mkdir(parents=True, exist_ok=True)


def copy_and_rename_image(row) -> str:
    old_value = row["old_image"]

    # пропуск пустых или невалидных значений
    if (
        not isinstance(old_value, str)
        or not old_value.strip()
        or (isinstance(old_value, float) and math.isnan(old_value))
    ):
        return None

    old_value = "." + old_value

    old_path = Path(old_value)
    print(old_path)
    if not old_path.exists():
        print(f"⚠️ Файл не найден: {old_path}")
        return None

    ext = old_path.suffix.lower()
    new_filename = f"{row['id']}{ext}"
    new_path = new_dir / new_filename

    shutil.copy2(old_path, new_path)
    return new_filename


# применяем к датафрейму
products["image_url"] = products.apply(copy_and_rename_image, axis=1)

upload\iblock\965\h6v1v60gi1ryckdh0tc7ay6ltnzwoi4a\2021-30-03_20-50-08__xDs6gko.webp
upload\iblock\da3\klvvib5jemjl1rp5rfraxiri73alqgtu\Снимок.PNG
upload\iblock\3bc\4b6qcru899yl23shyk2ndyfvic7eyqbp\s-l1600.jpg
upload\iblock\f18\n4qjnqyb01i5awupzy87xbd8w15z8dv5\Jobb-oldali-toloajto-zarszerkezet-ford-Transit-2000-2014-1766551-1-500x500.jpg
upload\iblock\30a\qpkbax69dk7oqivixfobnl5w4c062bw3\Изображение WhatsApp 2022-08-15 в 09.46.43.jpg
upload\iblock\e86\ygemqttik565m282bywo8ulfn6b9z8it\загружено.jpg
upload\iblock\452\r7wzzft4n0fqbatcvn54kvftuu28l81c\s-l1200.jpg
upload\iblock\12f\uq636hl13yzuptvz73t2ifek9j3fgpib\367_133-923.webp
upload\iblock\f23\6l5jhx8atvnyc2ywadu4j1bpdujgm9jp.jpeg
upload\iblock\f0e\9w1s02hi6ub3vped4y4unx3vyhgqvgyu.jpeg
upload\iblock\531\3lfp0z0644fcq5ezbcx7ju35v3886lj7\Снимок.PNG
upload\iblock\580\7r7ccj0be646o4j3oygfaigcm3vzq9ao\zl1rd78AC4jm.jpg
upload\iblock\155\47a8m6xnncpc20jkcsxqxt1oyof4e1ex\загружено.jpeg
upload\iblock\764\wv9ohc8az2nlnbdqn4bu31cotodys7p6\загруже

In [40]:
products["image_url"] = (
    "https://storage.yandexcloud.net/tcf-images/products/" + products["image_url"]
)
products["image_url"] = products["image_url"].fillna(
    "https://storage.yandexcloud.net/tcf-images/default.svg"
)
products["is_deleted"] = False
products.drop(columns=["old_image"]).to_csv("out/products.csv", index=False)

## Предложения (20 000 -> 25 000 за 4 месяца)
Скрестить с прайс-листом для цен и кол-ва

In [41]:
offers = pd.read_csv("data/offers.csv")

offers = offers[
    [
        "IE_XML_ID",
        "IE_NAME",
        "IE_PREVIEW_PICTURE",
    ]
].rename(
    columns={
        "IE_NAME": "name",
        "IE_XML_ID": "offer_bitrix_id",
        "IE_PREVIEW_PICTURE": "old_image",
    }
)

offers = offers[
    (offers["name"] != "1")
    & (offers["name"] != "0")
    & (offers["name"] != "01")
    & (offers["name"].notna())
]

In [42]:
offers

,offer_bitrix_id,name,old_image
0,F8-2,Focus II-III / C-Max / Kuga / Mondeo / S-Max 0...,/upload/iblock/d72/24sqmsxa5hpfyif8n4t4k5w1taf...
1,F8-3,Замок двери багажника Focus II-III,/upload/iblock/7b9/5vsxhfn3ve3rfwr3xm5fhcy63sy...
2,F8-4,Focus II-III / C-Max / Kuga / Mondeo / S-Max 0...,/upload/iblock/317/f1lpms9s3drhuewt5lyiwoyr1fd...
3,F8-5,Focus II-III / C-Max / Kuga / Mondeo / S-Max 0...,/upload/iblock/317/f1lpms9s3drhuewt5lyiwoyr1fd...
4,F8-6,Механизм замка Connect задней (высокой) двери ...,NaN
...,...,...,...
25043,378798,Transit 2.2TDCi 2008-2013 передняя RH (6 ст. V...,NaN
25044,378799,"Focus 2011- (ЛЕВАЯ, 26 шл. в 6-ступенч. КПП B6)",/upload/iblock/ba3/gglou8dovga4e8wcucxybdp1wug...
25045,378800,Taunus 1976-,/upload/iblock/59a/2wbuhb2ktei2ez6q62bcihd4k94...
25046,378801,1.6 Zetec-S Ti VCT (115-125PS) 2004-03.2006,/upload/iblock/9b6/7cmoh6wx23zgsngob0cepc4duka...


In [43]:
price_list = pd.read_excel("data/price_opt.xlsx")
price_list
# Проблемы в прайс-листе:
# - бренд - 5 пустых
# - Номер производителя - 19 пустых
# - цена опт - 27 пустых + 42, где цена = 1.0 руб
# - Того в прайсе 22,693 позиции 22,596 после чистки всех условий выше

,ID,Система,Подсистема,Название,Бренд,Номер производителя,Кроссы,Цена опт,Остаток
0,F2-123,Ford/Аксессуары,1,1,1,1,NaN,3609.0,0
1,F2-1,Ford/Аксессуары,Автомобильный набор,Автомобильный набор ARBORI Basic,AUTOFAMILY,AFEKBASIC01,AUTOFLEX NA0015,2408.0,0
2,F2-2,Ford/Аксессуары,Автомобильный набор,Автомобильный набор ARBORI Basic,AUTOFLEX,NA0015,AUTOFLEX NA0015,2284.0,0
3,F2-3,Ford/Аксессуары,Аптечка автомобильная,Аптечка,AIRLINE,AM04,NaN,253.0,0
4,F2-4,Ford/Аксессуары,Аптечка автомобильная,Аптечка,AIRLINE,AM01,NaN,630.0,0
...,...,...,...,...,...,...,...,...,...
22742,O1-185,Масла и автохимия/Тормозная жидкость,Тормозная жидкость,Жидкость тормозная Castrol BRAKE FLUID DOT 4 1Л,Castrol,15036B,"1776312, 1847947 - PFB401CN / 000000473402400;...",1033.0,3
22743,277112,Масла и автохимия/Тормозная жидкость,Тормозная жидкость,Жидкость тормозная DOT4 0.5 л. BOSCH,BOSCH,1987479106,000000473402400; 000000R403; 0009890807; 00098...,408.0,0
22744,278220,Масла и автохимия/Тормозная жидкость,Тормозная жидкость,Жидкость тормозная dot 4 - 0.5 VALEO 402402,VALEO\t\t,402402,000000473402400; 000000R403; 0009890807; 00098...,385.0,77
22745,276291,Масла и автохимия/Тормозная жидкость,Тормозная жидкость,ТОРМОЗНАЯ ЖИДКОСТЬ FORD 1847946 DOT 4,Ford,T207406,000000473402400; 000000R403; 0009890807; 00098...,619.0,0


In [124]:
price_list_clean = pd.read_excel("data/price_opt.xlsx")
price_list_clean = price_list_clean[
    [
        "ID",
        "Название",
        "Бренд",
        "Номер производителя",
        "Цена опт",
        "Цена розница",
        "Цена супер-опт",
        "Остаток",
        "bitrix_product_id",
        "bitrix_old_image"
    ]
]
price_list_clean["ID"] = price_list_clean["ID"].astype(str)
price_list_clean = price_list_clean[
    (price_list_clean["Название"] != "1")
    & (price_list_clean["Название"] != 1)
    & (price_list_clean["Название"] != "0")
    & (price_list_clean["Название"] != 0)
    & (price_list_clean["Название"] != "01")
    & (price_list_clean["Цена опт"] != 1.0)
    & (price_list_clean["Цена опт"].notna())
    & (price_list_clean["Бренд"].notna())
    & (price_list_clean["Номер производителя"].notna())
    & (price_list_clean["Название"].notna())
]

In [73]:
dirty = price_list[
    ~price_list["ID"].astype(str).isin(price_list_clean["ID"].astype(str))
]

In [76]:
dirty["link"] = "https://ford-parts.com.ru/catalog/" + dirty["ID"].astype(str) + "/"

C:\Users\utikpuhlik\AppData\Local\Temp\ipykernel_20752\301094796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dirty['link'] = "https://ford-parts.com.ru/catalog/" + dirty["ID"].astype(str) + "/"


In [77]:
dirty

,ID,Система,Подсистема,Название,Бренд,Номер производителя,Кроссы,Цена опт,Остаток,link
0,F2-123,Ford/Аксессуары,1,1,1,1,NaN,3609.0,0,https://ford-parts.com.ru/catalog/F2-123/
627,F4-143,Ford/Двигатель,Вкладыши коленвала,1.6 D 1984-1988 коренные STD,KOLBENSCHMIDT,87274600,NaN,NaN,0,https://ford-parts.com.ru/catalog/F4-143/
764,F4-289,Ford/Двигатель,Вкладыши коленвала,2.3 D / 2.5 D Sierra / Scorpio шатунные STD,AE,AEB4615,NaN,NaN,0,https://ford-parts.com.ru/catalog/F4-289/
851,F4-390,Ford/Двигатель,Вкладыши коленвала,Transit 1986-2000 2.5 D / TD коренные +0.75,AE,AEM5437,844F 6K371 DB T117394,1.0,0,https://ford-parts.com.ru/catalog/F4-390/
1123,F4-1372,Ford/Двигатель,К-т болтов головки,К-т болтов головки 1.8 CVH,GOETZE,22-18006B,NaN,NaN,0,https://ford-parts.com.ru/catalog/F4-1372/
...,...,...,...,...,...,...,...,...,...,...
22574,O1-411,Масла и автохимия/Моторные масла,Масло двигателя,1,NaN,NaN,NaN,NaN,0,https://ford-parts.com.ru/catalog/O1-411/
22601,O1-78,Масла и автохимия/Моторные масла,Масло двигателя,5W30 4 / 5Л,1,1,5W/30 M2C913D F4LTR N / 5W/30 M2C913D F7L N / ...,1.0,0,https://ford-parts.com.ru/catalog/O1-78/
22602,O1-80,Масла и автохимия/Моторные масла,Масло двигателя,5W30 4 / 5Л,1,1,5W/30 M2C913D F4LTR N / 5W/30 M2C913D F7L N / ...,1.0,0,https://ford-parts.com.ru/catalog/O1-80/
22626,O1-69,Масла и автохимия/Моторные масла,Масло двигателя,5л. VAG VW LongLife 3 5W30,1,1,5W/30 M2C913D F4LTR N / 5W/30 M2C913D F7L N / ...,1.0,0,https://ford-parts.com.ru/catalog/O1-69/


In [125]:
price_list_clean

,ID,Название,Бренд,Номер производителя,Цена опт,Цена розница,Цена супер-опт,Остаток,bitrix_product_id,bitrix_old_image
1,F2-1,Автомобильный набор ARBORI Basic,AUTOFAMILY,AFEKBASIC01,2408.0,2870.0,1945.0,0,F2-1,NaN
2,F2-2,Автомобильный набор ARBORI Basic,AUTOFLEX,NA0015,2284.0,2720.0,1847.0,0,F2-1,NaN
3,F2-3,Аптечка,AIRLINE,AM04,253.0,300.0,205.0,0,F2-3,NaN
4,F2-4,Аптечка,AIRLINE,AM01,630.0,750.0,509.0,0,F2-3,NaN
5,276085,Аптечка,AUTOFAMILY,TL8551,233.0,310.0,155.0,0,F2-3,/upload/iblock/090/i8cvqs13qnht9mp8k1kwfua0yjy...
...,...,...,...,...,...,...,...,...,...,...
22747,O1-185,Жидкость тормозная Castrol BRAKE FLUID DOT 4 1Л,Castrol,15036B,1033.0,1250.0,815.0,3,O1-184,/upload/iblock/172/6mo7kuys1ltq8m2pa1yr3yaedz6...
22748,277112,Жидкость тормозная DOT4 0.5 л. BOSCH,BOSCH,1987479106,408.0,500.0,315.0,0,O1-175,/upload/iblock/182/mx77ugi0o6duep680xu9g71htah...
22749,278220,Жидкость тормозная dot 4 - 0.5 VALEO 402402,VALEO\t\t,402402,385.0,450.0,319.0,77,O1-175,/upload/iblock/5be/1z48id2ioobistqzqfopcrs8wvm...
22750,276291,ТОРМОЗНАЯ ЖИДКОСТЬ FORD 1847946 DOT 4,Ford,T207406,619.0,740.0,497.0,0,O1-175,NaN


In [126]:
price_list_clean = price_list_clean.merge(
    products[["bitrix_id", "id"]],
    left_on=["bitrix_product_id"],
    right_on=["bitrix_id"],
    how="inner")
price_list_clean

,ID,Название,Бренд,Номер производителя,Цена опт,Цена розница,Цена супер-опт,Остаток,bitrix_product_id,bitrix_old_image,bitrix_id,id
0,F2-1,Автомобильный набор ARBORI Basic,AUTOFAMILY,AFEKBASIC01,2408.0,2870.0,1945.0,0,F2-1,NaN,F2-1,9e8e2d90-0cbe-4654-8713-612170bde364
1,F2-2,Автомобильный набор ARBORI Basic,AUTOFLEX,NA0015,2284.0,2720.0,1847.0,0,F2-1,NaN,F2-1,9e8e2d90-0cbe-4654-8713-612170bde364
2,F2-3,Аптечка,AIRLINE,AM04,253.0,300.0,205.0,0,F2-3,NaN,F2-3,94afe068-7066-4d55-bc1d-8e2011f1b62f
3,F2-4,Аптечка,AIRLINE,AM01,630.0,750.0,509.0,0,F2-3,NaN,F2-3,94afe068-7066-4d55-bc1d-8e2011f1b62f
4,276085,Аптечка,AUTOFAMILY,TL8551,233.0,310.0,155.0,0,F2-3,/upload/iblock/090/i8cvqs13qnht9mp8k1kwfua0yjy...,F2-3,94afe068-7066-4d55-bc1d-8e2011f1b62f
...,...,...,...,...,...,...,...,...,...,...,...,...
21957,O1-185,Жидкость тормозная Castrol BRAKE FLUID DOT 4 1Л,Castrol,15036B,1033.0,1250.0,815.0,3,O1-184,/upload/iblock/172/6mo7kuys1ltq8m2pa1yr3yaedz6...,O1-184,ca013479-8e13-4ac7-bc20-5077a58c2619
21958,277112,Жидкость тормозная DOT4 0.5 л. BOSCH,BOSCH,1987479106,408.0,500.0,315.0,0,O1-175,/upload/iblock/182/mx77ugi0o6duep680xu9g71htah...,O1-175,e74192ec-c3ea-4051-a79a-f75e5262f478
21959,278220,Жидкость тормозная dot 4 - 0.5 VALEO 402402,VALEO\t\t,402402,385.0,450.0,319.0,77,O1-175,/upload/iblock/5be/1z48id2ioobistqzqfopcrs8wvm...,O1-175,e74192ec-c3ea-4051-a79a-f75e5262f478
21960,276291,ТОРМОЗНАЯ ЖИДКОСТЬ FORD 1847946 DOT 4,Ford,T207406,619.0,740.0,497.0,0,O1-175,NaN,O1-175,e74192ec-c3ea-4051-a79a-f75e5262f478


In [127]:
price_list_clean = price_list_clean.rename(
    columns={
        "Бренд": "brand",
        "Номер производителя": "manufacturer_number",
        "Цена опт": "price_opt",
        "Цена розница": "wholesale_price_rub",
        "Цена супер-опт": "super_wholesale_price_rub",
        "Остаток": "quantity",
        "id": "product_id",
    }
)

In [128]:
price_list_clean

,ID,Название,brand,manufacturer_number,price_opt,wholesale_price_rub,super_wholesale_price_rub,quantity,bitrix_product_id,bitrix_old_image,bitrix_id,product_id
0,F2-1,Автомобильный набор ARBORI Basic,AUTOFAMILY,AFEKBASIC01,2408.0,2870.0,1945.0,0,F2-1,NaN,F2-1,9e8e2d90-0cbe-4654-8713-612170bde364
1,F2-2,Автомобильный набор ARBORI Basic,AUTOFLEX,NA0015,2284.0,2720.0,1847.0,0,F2-1,NaN,F2-1,9e8e2d90-0cbe-4654-8713-612170bde364
2,F2-3,Аптечка,AIRLINE,AM04,253.0,300.0,205.0,0,F2-3,NaN,F2-3,94afe068-7066-4d55-bc1d-8e2011f1b62f
3,F2-4,Аптечка,AIRLINE,AM01,630.0,750.0,509.0,0,F2-3,NaN,F2-3,94afe068-7066-4d55-bc1d-8e2011f1b62f
4,276085,Аптечка,AUTOFAMILY,TL8551,233.0,310.0,155.0,0,F2-3,/upload/iblock/090/i8cvqs13qnht9mp8k1kwfua0yjy...,F2-3,94afe068-7066-4d55-bc1d-8e2011f1b62f
...,...,...,...,...,...,...,...,...,...,...,...,...
21957,O1-185,Жидкость тормозная Castrol BRAKE FLUID DOT 4 1Л,Castrol,15036B,1033.0,1250.0,815.0,3,O1-184,/upload/iblock/172/6mo7kuys1ltq8m2pa1yr3yaedz6...,O1-184,ca013479-8e13-4ac7-bc20-5077a58c2619
21958,277112,Жидкость тормозная DOT4 0.5 л. BOSCH,BOSCH,1987479106,408.0,500.0,315.0,0,O1-175,/upload/iblock/182/mx77ugi0o6duep680xu9g71htah...,O1-175,e74192ec-c3ea-4051-a79a-f75e5262f478
21959,278220,Жидкость тормозная dot 4 - 0.5 VALEO 402402,VALEO\t\t,402402,385.0,450.0,319.0,77,O1-175,/upload/iblock/5be/1z48id2ioobistqzqfopcrs8wvm...,O1-175,e74192ec-c3ea-4051-a79a-f75e5262f478
21960,276291,ТОРМОЗНАЯ ЖИДКОСТЬ FORD 1847946 DOT 4,Ford,T207406,619.0,740.0,497.0,0,O1-175,NaN,O1-175,e74192ec-c3ea-4051-a79a-f75e5262f478


In [130]:
price_list_clean = price_list_clean.rename(
    columns={
        "Бренд": "brand",
        "Номер производителя": "manufacturer_number",
        "Цена опт": "price_opt",
        "Цена розница": "wholesale_price_rub",
        "Цена супер-опт": "super_wholesale_price_rub",
        "Остаток": "quantity",
        "id": "product_id",
    }
)

In [131]:

offers = price_list_clean[
    [
        "product_id",
        "brand",
        "manufacturer_number",
        "price_opt",
        "wholesale_price_rub",
        "super_wholesale_price_rub",
        "quantity",
        "bitrix_old_image",
    ]].copy()

In [132]:
ids = [str(uuid4()) for i in range(len(offers))]
offers['id'] = ids

In [133]:
offers

,product_id,brand,manufacturer_number,price_opt,wholesale_price_rub,super_wholesale_price_rub,quantity,bitrix_old_image,id
0,9e8e2d90-0cbe-4654-8713-612170bde364,AUTOFAMILY,AFEKBASIC01,2408.0,2870.0,1945.0,0,NaN,2e242e32-2c5b-40ed-b413-db9d48dcf145
1,9e8e2d90-0cbe-4654-8713-612170bde364,AUTOFLEX,NA0015,2284.0,2720.0,1847.0,0,NaN,a7419bae-284f-452b-b903-bb836c3cf615
2,94afe068-7066-4d55-bc1d-8e2011f1b62f,AIRLINE,AM04,253.0,300.0,205.0,0,NaN,311c81ce-15c8-4c4e-a75f-8f4c3c4ac3e4
3,94afe068-7066-4d55-bc1d-8e2011f1b62f,AIRLINE,AM01,630.0,750.0,509.0,0,NaN,f923c1a3-920d-4172-919d-10fec07d6f7d
4,94afe068-7066-4d55-bc1d-8e2011f1b62f,AUTOFAMILY,TL8551,233.0,310.0,155.0,0,/upload/iblock/090/i8cvqs13qnht9mp8k1kwfua0yjy...,98647981-0896-4d64-b657-54e1834fa25d
...,...,...,...,...,...,...,...,...,...
21957,ca013479-8e13-4ac7-bc20-5077a58c2619,Castrol,15036B,1033.0,1250.0,815.0,3,/upload/iblock/172/6mo7kuys1ltq8m2pa1yr3yaedz6...,dcb156d2-f7b6-4efc-81e9-f73733e021b4
21958,e74192ec-c3ea-4051-a79a-f75e5262f478,BOSCH,1987479106,408.0,500.0,315.0,0,/upload/iblock/182/mx77ugi0o6duep680xu9g71htah...,6c6c156a-a9bd-4ddf-9ab5-570ac118acc9
21959,e74192ec-c3ea-4051-a79a-f75e5262f478,VALEO\t\t,402402,385.0,450.0,319.0,77,/upload/iblock/5be/1z48id2ioobistqzqfopcrs8wvm...,930c3f1b-c5bf-4fff-96ee-beae6be7ae45
21960,e74192ec-c3ea-4051-a79a-f75e5262f478,Ford,T207406,619.0,740.0,497.0,0,NaN,68ccf37a-f6cb-47ab-8e91-0d26d8fde788


In [136]:
old_dir = Path("./upload")
new_dir = Path("./offers")
new_dir.mkdir(parents=True, exist_ok=True)


def copy_and_rename_image(row) -> str:
    old_value = row["bitrix_old_image"]

    # пропуск пустых или невалидных значений
    if (
        not isinstance(old_value, str)
        or not old_value.strip()
        or (isinstance(old_value, float) and math.isnan(old_value))
    ):
        return None

    old_value = "." + old_value

    old_path = Path(old_value)
    print(old_path)
    if not old_path.exists():
        print(f"⚠️ Файл не найден: {old_path}")
        return None

    ext = old_path.suffix.lower()
    new_filename = f"{row['id']}{ext}"
    new_path = new_dir / new_filename

    shutil.copy2(old_path, new_path)
    return new_filename


# применяем к датафрейму
offers["image_url"] = offers.apply(copy_and_rename_image, axis=1)

upload\iblock\090\i8cvqs13qnht9mp8k1kwfua0yjypt2bf\загружено (1).jpg
upload\iblock\1e2\l09ngy47yzbocru27r6be2b41idk7vbm\Изображение WhatsApp 2023-02-03 в 13.33.20.jpg
upload\iblock\fd8\9uxoqeyms6dbmz5kcbwxmsrhcl848uhq\Изображение WhatsApp 2023-02-03 в 13.22.42.jpg
upload\iblock\102\gyks1lr91tgcly8z01uq0qrdbnc7xk2y\Изображение WhatsApp 2023-02-03 в 13.26.38.jpg
upload\iblock\599\bln4bdkc3tdreya6a5930te9xwh1cz3x\Изображение WhatsApp 2023-02-03 в 13.28.26.jpg
upload\iblock\e05\a9nz4kn0q2tqi1irj53lnl15i5suu1my\Изображение WhatsApp 2023-02-03 в 13.30.27.jpg
upload\iblock\d44\ghwn0cju7qx35zwcs8pevuq2853owspm\O1CN01kt0SjH1yHKMri57hM_!!2216763496553-0-cib.jpg
upload\iblock\053\s5csz4cvzjqcpmf2164qvuxo4z7kjtbb\Снимок экрана 2024-06-12 171126.png
upload\iblock\ab2\ofrf503gnb8p1a8z1sy96k0alivcl2vb\voy1438_1.jpg
upload\iblock\f29\xtdy2ewu388ork21ya1bi3cd0qzm7wuc\загружено (2).jpg
upload\iblock\89a\t833fso8sd18kznpujd6x4ugalnr3e0e\загружено (2).jpeg
upload\iblock\11b\sa5txq7cx53fisveqwnsthivksbhdn0

In [137]:
offers["image_url"] = (
    "https://storage.yandexcloud.net/tcf-images/offers/" + offers["image_url"]
)
offers["image_url"] = offers["image_url"].fillna(
    "https://storage.yandexcloud.net/tcf-images/default.svg"
)
offers["is_deleted"] = False
offers.drop(columns=["bitrix_old_image"]).to_csv("out/offers.csv", index=False)